White box. Run mwsat :

In [ ]:
import os
import subprocess

directories = [
    "wuf20-71R/wuf20-71R-M/",
    "wuf20-71R/wuf20-71R-N/",
    "wuf20-71R/wuf20-71R-Q/",
    "wuf20-71R/wuf20-71R-R/",

    "wuf20-91R/wuf20-91R-M/",
    "wuf20-91R/wuf20-91R-N/",
    "wuf20-91R/wuf20-91R-Q/",
    "wuf20-91R/wuf20-91R-R/",

    "wuf50-218R/wuf50-218R-M/",
    "wuf50-218R/wuf50-218R-N/",
    "wuf50-218R/wuf50-218R-Q/",
    "wuf50-218R/wuf50-218R-R/"
]

for out_dir in directories:
    os.makedirs(os.path.join("out", out_dir), exist_ok=True)

NUM_RUNS = 100

for input_dir in directories:
    full_input_dir = os.path.join("data", input_dir)
    if os.path.isdir(full_input_dir):
        for file in os.listdir(full_input_dir):
            if file.endswith(".mwcnf"):
                file_path = os.path.join(full_input_dir, file)
                for _ in range(NUM_RUNS):
                    subprocess.run(["./mwsat", file_path])
    else:
        print(f"Folder {full_input_dir} not found!")


White box. Evaluate average steps, success and optimal rates :

In [ ]:
import os
import numpy as np

directories = [
    "wuf20-71R/wuf20-71R-M",
    "wuf20-71R/wuf20-71R-N",
    "wuf20-71R/wuf20-71R-Q",
    "wuf20-71R/wuf20-71R-R",

    "wuf20-91R/wuf20-91R-M",
    "wuf20-91R/wuf20-91R-N",
    "wuf20-91R/wuf20-91R-Q",
    "wuf20-91R/wuf20-91R-R",

    "wuf50-218R/wuf50-218R-M",
    "wuf50-218R/wuf50-218R-N",
    "wuf50-218R/wuf50-218R-Q",
    "wuf50-218R/wuf50-218R-R"
]


def load_optimal_values(opt_file):
    optimal_values = {}
    with open(opt_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            instance_name = parts[0]
            opt_value = int(parts[1])
            optimal_values[instance_name] = opt_value
    return optimal_values


def evaluate_results(results_dir, optimal_values):
    match_counts = []

    for filename in os.listdir(results_dir):
        if filename.endswith(".dat"):
            instance_name = filename.replace("wuf", "uf").replace(".dat", "")
            if instance_name in optimal_values:
                opt_value = optimal_values[instance_name]

                with open(os.path.join(results_dir, filename), 'r') as f:
                    results = []
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) > 3:
                            try:
                                results.append(int(parts[1]))
                            except ValueError:
                                print(f"Conversion error: {line.strip()}")
                        else:
                            results.append(0)

                if results:
                    match_count = sum(1 for res in results if res == opt_value)
                    match_counts.append(match_count / len(results))

    return np.mean(match_counts) if match_counts else 0.0


def evaluate_success_and_steps(results_dir):
    total_problems = 0
    solved_problems = 0
    total_steps = 0
    steps_count = 0

    wuf_file = os.path.join(results_dir, "merged_wuf.dat")
    with open(wuf_file, 'r') as f:
        for line in f:
            total_problems += 1
            if len(line.split()) > 3:
                solved_problems += 1

    iter_file = os.path.join(results_dir, "merged_iter.dat")
    with open(iter_file, 'r') as f:
        for line in f:
            steps = int(line.split()[1])
            total_steps += steps
            steps_count += 1

    success_rate = (solved_problems / total_problems) * 100 if total_problems > 0 else 0
    avg_steps = total_steps // steps_count if steps_count > 0 else 0
    return success_rate, avg_steps


def main():
    base_data_dir = "data"
    base_results_dir = "out"

    print("Instances | Success Rate (%) | Optimal Rate (%) | Average Steps")

    for dir_name in directories:
        opt_file = os.path.join(base_data_dir, f"{dir_name}-opt.dat")
        results_dir = os.path.join(base_results_dir, dir_name)

        if os.path.exists(opt_file) and os.path.exists(results_dir):
            optimal_values = load_optimal_values(opt_file)
            optimal_rate = evaluate_results(results_dir, optimal_values)

            success_rate, avg_steps = evaluate_success_and_steps(results_dir)
            print(f"{dir_name} | {success_rate:.2f}% | {optimal_rate * 100:.2f}% | {avg_steps}")
        else:
            print(f"Skipping {dir_name}, missing files")


main()
